In [1]:
import os
os.chdir('..')

from tqdm import tqdm

import numpy as np
import pandas as pd
import scipy
from scipy.stats import t

from bokeh.plotting import show
from bokeh.models.annotations import Title
from bokeh.models import Plot, ColumnDataSource, Ellipse, Circle, Grid, LinearAxis, Text
from bokeh.io import output_notebook, export_png

# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import matplotlib.transforms as transforms
# from matplotlib.patches import Ellipse

from puncta_counter.src.preprocessing import preprocess_df
from puncta_counter.src.summarize import generate_circle, generate_ellipse
from puncta_counter.utils.ellipse_algos import confidence_ellipse, min_vol_ellipse

pd.options.display.max_columns = None
output_notebook()

Loading BokehJS ...

In [2]:
def instantiate_plot_bokeh(
    title=None,
    fig_width=1000,
    fig_height=800,
    toolbar_location=None,
    xaxis_position='above',
    yaxis_position='left',
    x_range=[0, 1290],
    y_range=[1000, 0],
):
    plot = Plot(
        title=Title(text=title),
        width=fig_width, height=fig_height,
        match_aspect=True,
        toolbar_location=toolbar_location
    )

    # plot.title = Title(text=title)
    xaxis = LinearAxis()
    plot.add_layout(xaxis, xaxis_position)
    plot.x_range.start = x_range[0]
    plot.x_range.end = x_range[1]

    yaxis = LinearAxis()
    plot.add_layout(yaxis, yaxis_position)
    plot.y_range.start = y_range[0]
    plot.y_range.end = y_range[1]

    plot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
    plot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))
    
    return plot

In [3]:
def plot_ellipse_using_bokeh(
        column_data,
        text_data=None,
        x='x',
        y='y',
        height="height",
        width="width",
        angle='angle',
        text=None,
        title=None,
        fill_color='#000fff',  # blue
        text_color='white',
        line_alpha=1.2,
        line_width=0,
        plot=None,
    ):
    
    """Data should be a dataframe, for example:
    df[['x', 'y', 'height', 'width', 'angle', 'text']]
    """
    
    if plot is None:
        plot=instantiate_plot_bokeh()
        
    if title:
        plot.title = Title(text=title)

    column_data_source = ColumnDataSource(column_data)
        
    ellipse_glyph = Ellipse(
        x=x, y=y, width=width,
        height=height, angle=angle,
        line_color='#FFFFFF',
        fill_color=fill_color,
        line_width=line_width,
        line_alpha=line_alpha,
    )
    plot.add_glyph(column_data_source, ellipse_glyph)
    
    if text_data is not None:
        text_data_source = ColumnDataSource(text_data)
        text_glyph = Text(
            x=x, y=y,
            text=text,
            text_color=text_color,
            text_font_size = {'value': '13px'}
        )
        plot.add_glyph(text_data_source, text_glyph)
    
    return plot

In [4]:
def plot_circle_using_bokeh(
        column_data,
        text_data=None,
        x='x',
        y='y',
        size="size",
        text=None,
        title=None,
        fill_color='#000fff',  # blue
        text_color='white',
        line_alpha=1.2,
        line_width=0,
        plot=None,
    ):
    
    """Data should be a dataframe, for example:
    df[['x', 'y', 'size' 'text']]
    """
    
    if plot is None:
        plot=instantiate_plot_bokeh()
        
    if title:
        plot.title = Title(text=title)

    column_data_source = ColumnDataSource(column_data)
        
    circle_glyph = Circle(
        x=x, y=y,
        size=size,
        line_color='#FFFFFF',
        fill_color=fill_color,
        line_width=line_width,
        line_alpha=line_alpha,
    )
    plot.add_glyph(column_data_source, circle_glyph)
    
    if text_data is not None:
        text_data_source = ColumnDataSource(text_data)
        text_glyph = Text(
            x=x, y=y,
            text=text,
            text_color=text_color,
            text_font_size = {'value': '13px'}
        )
        plot.add_glyph(text_data_source, text_glyph)
    
    return plot

# Data

In [5]:
# read in data
nuclei = pd.read_csv("data/nuclei_subset.csv")
nuclei['angle'] = nuclei['orientation'].apply(lambda x: x/180*np.pi)

puncta = pd.read_csv("data/puncta_subset.csv")

In [6]:
# filters
nuclei_subset = nuclei[
    (nuclei['eccentricity'] < 0.69)
    & (nuclei['major_axis_length'] < 128)
].copy()

puncta = pd.merge(
    left=nuclei_subset[["image_number", 'object_number']],
    right=puncta.loc[:, puncta.columns != 'object_number'],
    left_on=["image_number", 'object_number'],
    right_on=['image_number', 'nuclei_object_number'],
    how="left",
).dropna(subset=['nuclei_object_number'])  # left join without duplicates
puncta['angle'] = puncta['orientation'].apply(lambda x: x/180*np.pi)

In [7]:
filename_for_image_number = dict(zip(nuclei_subset['image_number'], nuclei_subset['file_name_tif']))

# Plot

In [8]:
circles = generate_circle(puncta)

In [9]:
def min_vol_ellipse(P, tolerance=0.05, **kwargs):
    # See: https://www.mathworks.com/matlabcentral/fileexchange/9542-minimum-volume-enclosing-ellipsoid
    # Original author: Nima Moshtagh (nima@seas.upenn.edu)
    # Translated to Python by Harrison Wang


    # data points
    # -----------------------------------
    
    d, N = P.shape
    if N <= d:
        return np.nan, np.nan, np.nan, np.nan, np.nan
    
    # Q = np.zeros((d+1,N))
    # Q(1:d,:) = P(1:d,1:N)
    # Q(d+1,:) = np.ones(1,N)
    Q = np.vstack([P, np.ones((1, N))])

    # initializations
    # -----------------------------------
    count = 1
    err = 1
    u = (1/N) * np.ones((N, 1))  # 1st iteration
    
    # Khachiyan Algorithm
    # -----------------------------------
    while err > tolerance:
        try:
            X = np.dot(np.dot(Q, u* np.identity(N)), np.transpose(Q))
            M = np.diag(np.dot(np.dot(np.transpose(Q), np.linalg.inv(X)), Q))  # M the np.diagonal vector of an NxN matrix

            j = np.argmax(M)
            maximum = max(M)

            step_size = (maximum - d -1)/((d+1)*(maximum-1))
            new_u = (1 - step_size)*u 
            new_u[j] = new_u[j] + step_size

            count = count + 1
            err = np.linalg.norm(new_u - u)
            u = new_u
        except:
            print(P)
            break
                    
    ################### Computing the Ellipse parameters######################
    # Finds the ellipse equation in the 'center form': 
    # (x-c)' * A * (x-c) = 1
    # It computes a dxd matrix 'A' and a d dimensional vector 'c' as the center
    # of the ellipse. 
    U = u * np.identity(N)
    # return P, u
    
    # the A matrix for the ellipse
    # A = (1/d) * inv(P * U * P' - (P * u)*(P*u)' );
    # --------------------------------------------
    A = (1/d) * np.linalg.inv(
        np.dot(np.dot(P, U), np.transpose(P)) - np.dot(np.dot(P, u), np.transpose(np.dot(P, u)))
    )
    
    # center of the ellipse 
    # --------------------------------------------
    c = np.dot(P, u)
    
    # original return value
    # return A, c
    
    center_x = c[0][0]
    center_y = c[1][0]

    # bounding box
    inv_A = np.linalg.inv(A)  # diagonals are the important terms

    min_x = center_x - np.sqrt(inv_A[0][0])
    max_x = center_x + np.sqrt(inv_A[0][0])
    min_y = center_y - np.sqrt(inv_A[1][1])
    max_y = center_y + np.sqrt(inv_A[1][1])
    
    return center_x, center_y, max_x-min_x, max_y-min_y, 0


In [10]:
puncta['center'] = puncta[['center_x', 'center_y']].apply(list, axis=1)
puncta['total_intensity'] = puncta["integrated_intensity"] * puncta["area"]

puncta_summary = (
    puncta
    .groupby(['image_number', 'object_number'])[['center', "total_intensity"]]
    .agg(list)
    .reset_index()
).copy()


(puncta_summary["center"]
        .apply(lambda x: np.transpose(np.array(x)))
        .apply(lambda x: min_vol_ellipse(x, tolerance=0.05))
    )[0]

(331.0440274394413,
 61.90365017447911,
 13.562292742663203,
 17.15741308323333,
 0)

In [11]:
puncta_summary

,image_number,object_number,center,total_intensity
0,1,4,"[[330.33333333333303, 52.6666666666666], [329....","[0.231494620442391, 0.7203021273016901, 80.982..."
1,1,6,"[[481.0, 108.0], [476.75, 118.0], [488.3783783...","[0.023727778345346, 0.415106430649756, 43.6711..."
2,1,7,"[[633.4, 106.2], [623.2, 108.6], [634.8, 109.6...","[0.6460669729858599, 0.6246280577033749, 0.643..."
3,1,8,"[[256.666666666666, 105.333333333333], [261.0,...","[0.223071645945312, 2.223804071545596, 0.09936..."
4,1,9,"[[522.6, 119.2], [520.5, 120.0], [525.66666666...","[0.617074845358725, 0.09710841625928801, 2.053..."
...,...,...,...,...
406,20,49,"[[661.5, 867.5]]",[0.364629589021204]
407,20,50,"[[132.0, 913.0], [140.875, 918.125], [144.1666...","[0.022293431684375, 1.53884182870388, 0.907942..."
408,20,51,"[[1056.0, 899.5], [1062.33333333333, 900.33333...","[0.09198138490319198, 0.224261848255992, 9.682..."
409,20,52,"[[260.5, 866.0], [272.0, 876.0], [264.0, 877.0...","[0.08990615606308, 0.021881436929107, 0.023010..."


In [12]:
circles

,image_number,nuclei_object_number,area_sum,area_count,integrated_intensity_sum,center_x_mean,center_x_std,center_y_mean,center_y_std,center_std,effective_radius_puncta,bounding_box_min_x,bounding_box_max_x,bounding_box_min_y,bounding_box_max_y
0,1,4.0,160.0,11,4.902541,330.989711,4.129496,60.479130,5.353243,6.760913,12.748500,318.241210,343.738211,47.730630,73.227630
1,1,6.0,224.0,20,6.484916,481.612065,6.820752,128.712632,7.670303,10.264317,19.354582,462.257483,500.966646,109.358050,148.067214
2,1,7.0,45.0,12,1.132769,628.586111,6.101142,112.444444,3.523151,7.045319,13.284781,615.301330,641.870893,99.159663,125.729226
3,1,8.0,15.0,4,0.394888,258.916667,3.023060,108.208333,2.096624,3.678957,6.937107,251.979559,265.853774,101.271226,115.145441
4,1,9.0,74.0,14,1.917098,525.746250,4.705514,124.630795,3.997413,6.174235,11.642249,514.104001,537.388499,112.988546,136.273045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,20,49.0,4.0,1,0.091157,661.500000,NaN,867.500000,NaN,NaN,1.128379,660.371621,662.628379,866.371621,868.628379
407,20,50.0,37.0,10,0.870710,142.769881,7.776392,923.256310,6.621799,10.213740,19.259213,123.510668,162.029094,903.997096,942.515523
408,20,51.0,175.0,16,4.513405,1060.745804,3.282882,907.430531,5.931118,6.779046,12.782693,1047.963111,1073.528497,894.647838,920.213224
409,20,52.0,19.0,15,0.433860,256.533333,14.516575,889.666667,11.482388,18.508814,34.900554,221.632779,291.433887,854.766113,924.567221


In [13]:
puncta_summary = generate_ellipse(puncta, algo='confidence_ellipse')
puncta_summary['angle'] = puncta_summary['orientation'].apply(lambda x: x/180*np.pi)

In [14]:
# data

image_number=3

title = filename_for_image_number[3]

nuclei_data = nuclei_subset.loc[
    (nuclei_subset['image_number']==image_number),
    ["object_number", "center_x", "center_y", "major_axis_length", "minor_axis_length", "angle"]
]

puncta_summary_data = puncta_summary.loc[
    (puncta_summary['image_number']==image_number),
    ["object_number", "center_x", "center_y", "major_axis_length", "minor_axis_length", "angle"]
]

puncta_data = puncta.loc[
    (puncta['image_number']==image_number),
    ["object_number", "center_x", "center_y", "major_axis_length", "minor_axis_length", "angle"]
]


# nuclei
plot = plot_circle_using_bokeh(
    nuclei_data,
    nuclei_data,
    x='center_x',
    y='center_y',
    size="major_axis_length",
    text="object_number",
    title=title,
    fill_color='#000fff',  # blue
)

# # puncta_summary
# plot = plot_ellipse_using_bokeh(
#     puncta_summary_data,
#     puncta_summary_data,
#     x='center_x',
#     y='center_y',
#     height="major_axis_length",
#     width="minor_axis_length",
#     angle='angle',
#     text="object_number",
#     text_color='orange',
#     fill_color='#097969',  # green
#     line_alpha=0,
#     plot=plot
# )

# # puncta
# plot = plot_ellipse_using_bokeh(
#     puncta_data,
#     x='center_x',
#     y='center_y',
#     height="major_axis_length",
#     width="minor_axis_length",
#     angle='angle',
#     fill_color='#ff2b00',  # red
#     line_alpha=0,
#     plot=plot
# )

In [15]:
show(plot)